<a href="https://colab.research.google.com/github/rabbitmetrics/cx-analytics/blob/main/notebooks/cx-analytics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
# Link to the data https://cseweb.ucsd.edu/~jmcauley/datasets/amazon_v2/

In [8]:
# !pip install python-dotenv==1.0.0
# !pip install pandas==2.0.1
# !pip install langchain==0.0.137
# !pip install openai==0.27.5
# !pip install pinecone-client==2.2.1
# !pip install tqdm==4.65.0
# !pip install sentence-transformers==2.2.2
# !pip install scikit-learn==1.2.2
# !pip install llama-cpp-python==0.1.48
# !pip install huggingface_hub==0.14.1
# !pip install sentence_transformers==2.2.2
# !pip install faiss-gpu==1.7.2
# !pip install protobuf==3.20.3

In [ ]:
# Login to huggingface hub
from huggingface_hub import notebook_login
notebook_login()

In [10]:
import os
import json
import gzip
import pandas as pd
from urllib.request import urlopen

from dotenv import load_dotenv,find_dotenv

load_dotenv(find_dotenv())

False

In [11]:
# If using Google Colab, mount your Google Drive
# from google.colab import drive
# drive.mount("google_drive")

In [12]:
# Extract data from files
data = []
with gzip.open('data/Amazon_Fashion.json.gz') as f:
    for l in f:
        data.append(json.loads(l.strip()))
        
metadata = []
with gzip.open('data/meta_Amazon_Fashion.json.gz') as f:
    i = 0
    for l in f:
      try:
        j = json.loads(l.strip())
        metadata.append(j)
      except:
        i += 1
        continue

print("total load data:", len(data), ", total load meta data:", len(metadata), ", ignored meta data:", i)

total load data: 83497 , total load meta data: 23950 , ignored meta data: 195


In [13]:
print(data[0])
print(metadata[0])

{'reviewerID': 'A2LOTV0TWU6UQI', 'asin': '1880241196', 'reviewerName': 'Patricia Vincent', 'helpful': [4, 4], 'reviewText': "I ordered this for Christmas for my grandson along with a pair of binoculars.I thought I was ordering a book and was surprised by this thin booklet.Its written for adults not children, more for taking your children out looking for birds.I am not returning it because its now too close to Christmas to order anything else.Just letting any reader know, its a small booklet, not a book and it's not something that would be easy to read by children.I wish I had read the whole description.", 'overall': 1.0, 'summary': 'Did not read the whole paragragh', 'unixReviewTime': 1355270400, 'reviewTime': '12 12, 2012'}
{'asin': '1880241196', 'title': "Bird Watching For Kids - A Special Publication from Bird Watcher's Digest", 'price': 4.5, 'imUrl': 'http://ecx.images-amazon.com/images/I/51GhsVrjLCL._SY300_.jpg', 'salesRank': {'Watches': 54623}, 'categories': 'Amazon Fashion'}


In [14]:
# Load the data to dataframes

df = pd.DataFrame.from_dict(data)
df = df[df['reviewText'].notna()]

df_meta=pd.DataFrame.from_dict(metadata)

In [15]:
df.head(3)

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A2LOTV0TWU6UQI,1880241196,Patricia Vincent,"[4, 4]",I ordered this for Christmas for my grandson a...,1.0,Did not read the whole paragragh,1355270400,"12 12, 2012"
1,A1H1L1MZXLMGJ3,3890968740,"Shannon Farms ""Children's Library Assistant""","[0, 0]",I got this watch for my 13 year old daughter. ...,4.0,So far still ticking away,1362182400,"03 2, 2013"
2,A2H3F19K13QQOX,5555014597,Cindy Littlejohn,"[0, 0]","3rd year running I have brought these, takes a...",5.0,"3rd year purchased, always good hats!",1357257600,"01 4, 2013"


In [16]:
df_meta.head(3)

,asin,title,price,imUrl,salesRank,categories,brand,description
0,1880241196,Bird Watching For Kids - A Special Publication...,4.5,http://ecx.images-amazon.com/images/I/51GhsVrj...,{'Watches': 54623},Amazon Fashion,NaN,NaN
1,3890968740,NEW Mango Yellow Mix Colorful Jelly Silicone S...,NaN,http://ecx.images-amazon.com/images/I/51cDlNPM...,{'Watches': 199658},[['Amazon Fashion']],NaN,NaN
2,5555014597,Top Gear Official Merchandise - Top Gear Baseb...,19.5,http://ecx.images-amazon.com/images/I/41PAOE%2...,{'Clothing': 296726},Amazon Fashion,NaN,NaN


In [17]:
# Truncate the reviewText

max_text_length=400
def truncate_review(text):
    return text[:max_text_length]

df['truncated']=df.apply(lambda row: truncate_review(row['reviewText']),axis=1)

In [18]:
# Look for productIds with enough reviews

df.groupby('asin').count().sort_values('overall')

,reviewerID,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,truncated
asin,,,,,,,,,
1880241196,1,1,1,1,1,1,1,1,1
B006IABUNW,1,1,1,1,1,1,1,1,1
B006ICMFH0,1,1,1,1,1,1,1,1,1
B006IFHTKU,1,1,1,1,1,1,1,1,1
B006IFLEB0,1,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...
B008HZYCEU,380,380,380,380,380,380,380,380,380
B001ULD9YC,392,391,392,392,392,392,392,392,392
B00272N9PW,491,491,491,491,491,491,491,491,491


In [19]:
# Work on only a slice of the dataframe

df_review = df.loc[df['asin'] == 'B004B8C2CQ'].copy()

In [20]:
len(df_review)

719

## Now applying AI models to analyze the data

In [21]:
# Set huggingface cache directory if necessary
import sys, os

hf_home = "/mnt/e/huggingface/cache"
os.environ['HF_HOME'] = hf_home
os.environ['HUGGINGFACE_HUB_CACHE'] = hf_home + "/hub"
os.environ['SENTENCE_TRANSFORMERS_HOME'] = hf_home + "/torch/sentence_transformers"

In [22]:
# Using HuggingFaceEmbeddings

from langchain.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings()

In [28]:
df_review['embeddings']=df_review.apply(lambda row: embeddings.embed_query(row['truncated']), axis=1)
df_review.head(3)

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,truncated,embeddings
37582,A26QHY5FHZ1HGO,B004B8C2CQ,A. Ahmed,"[0, 0]","It is a great material, great color! and a goo...",5.0,"great material, great color!",1358640000,"01 20, 2013","It is a great material, great color! and a goo...","[0.012763723731040955, 0.05646521970629692, -0..."
37583,A1UHYD0RBA0AVM,B004B8C2CQ,aak,"[0, 0]",I purchased the blue and white colorway and I ...,5.0,Looks amazing,1344643200,"08 11, 2012",I purchased the blue and white colorway and I ...,"[0.005255925469100475, 0.05292578414082527, 0...."
37584,A1RC0SK3X55PQK,B004B8C2CQ,Aaron Don,"[0, 0]",It didn't feel like its supposed to be warm li...,3.0,yeah shemagh,1390867200,"01 28, 2014",It didn't feel like its supposed to be warm li...,"[-0.0018896874971687794, -0.030937310308218002..."


In [29]:
df_meta[df_meta.asin=='B004B8C2CQ']

,asin,title,price,imUrl,salesRank,categories,brand,description
9419,B004B8C2CQ,Premium Shemagh Head Neck Scarf by Tapp Collec...,11.99,http://ecx.images-amazon.com/images/I/51ou0U3H...,{'Clothing': 219},Amazon Fashion,NaN,NaN


In [25]:
# Prepare training and test sets for training Random Forest Regressor

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

X_train, X_test, y_train, y_test = train_test_split(
    list(df_review.embeddings.values),
    df_review.overall,
    test_size = 0.2,
    random_state=1
)


In [26]:
# Train and calculate mean absolute error
import numpy as np
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(n_estimators=150)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
rmae = np.sqrt(mae)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
print("Root Mean Absolute Error:", rmae, ", Root Mean Squared Error:", rmse)

Root Mean Absolute Error: 0.8756214724199967 , Root Mean Squared Error: 0.9171559300358909


In [27]:
# Create list with truncated review texts

texts=df['truncated'].tolist()

## Create the Vectorstore

### Option 1. Use Pinecone as the vectorstore

In [57]:
# Import Pinecone client

import pinecone
from langchain.vectorstores import Pinecone

# Initialize Pinecone
pinecone.init(
    api_key= os.getenv('PINECONE_API_KEY'),  
    environment= os.getenv('PINECONE_ENV')  
)

In [ ]:
# Send embedding vectors to Pinecone with Langchain

vstore = Pinecone.from_texts(texts, embeddings, index_name='cxanalytics')

In [ ]:
# Do a basic vector similarity search

query = "poor product"
result = vstore.similarity_search(query)
print(result)

### Option 2. Use FAISS as the vectorstore

In [30]:
from langchain.vectorstores import FAISS
faiss_vstore = FAISS.from_texts(texts, embeddings)

In [37]:
# Do a basic vector similarity search

query = "poor product"
result = faiss_vstore.similarity_search(query)
print(result)

[Document(page_content='Poor quality', metadata={}), Document(page_content='Poor quality.', metadata={}), Document(page_content='Poor quality.', metadata={}), Document(page_content='Great design. Poor quality', metadata={})]


In [32]:
vstore = faiss_vstore

## Use RetrievalQA to interact with AI
### Option 1. Use ChatGPT

In [ ]:
# Import RetrievalQA adn ChatOpenAPI and define review_chain in order to have GPT-4 access the review data

from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

chat = ChatOpenAI(model_name="gpt-4",temperature=0.0)
review_chain = RetrievalQA.from_chain_type(llm=chat, chain_type="stuff", retriever=vstore.as_retriever())

### Option2. Use HuggingFace Local AI model

In [46]:
# Import RetrievalQA adn HuggingFacePipeline to load local models
from langchain.llms import HuggingFacePipeline
from langchain.llms import LlamaCpp

# Use HuggingFace online model. However my 3090 GPU can't load it successfully.
# from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
# model_id = "TheBloke/vicuna-7B-1.1-HF"
# tokenizer = AutoTokenizer.from_pretrained(model_id)
# model = AutoModelForCausalLM.from_pretrained(model_id)
# pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

# Or download the quantizied model to local disk ./models/ 
# And load it via LlamaCpp
llm = LlamaCpp(model_path="models/ggml-vicuna-13b-1.1-q4_2.bin", n_ctx=2048)

llama.cpp: loading model from models/ggml-vicuna-13b-1.1-q4_2.bin
llama_model_load_internal: format     = ggjt v1 (latest)
llama_model_load_internal: n_vocab    = 32000
llama_model_load_internal: n_ctx      = 2048
llama_model_load_internal: n_embd     = 5120
llama_model_load_internal: n_mult     = 256
llama_model_load_internal: n_head     = 40
llama_model_load_internal: n_layer    = 40
llama_model_load_internal: n_rot      = 128
llama_model_load_internal: ftype      = 5 (mostly Q4_2)
llama_model_load_internal: n_ff       = 13824
llama_model_load_internal: n_parts    = 1
llama_model_load_internal: model size = 13B
llama_model_load_internal: ggml ctx size =  85.08 KB
llama_model_load_internal: mem required  = 9807.48 MB (+ 3216.00 MB per state)
AVX = 1 | AVX2 = 1 | AVX512 = 1 | AVX512_VBMI = 1 | AVX512_VNNI = 1 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 0 | SSE3 = 1 | VSX = 0 | 
llama_init_from_file: kv self size  = 3200.00 MB


In [47]:
# Define the task for GPT-4 and run the chain
from langchain.chains import RetrievalQA

q="""The reviews you see are for a product called 'Premium Shemagh Head Neck Scarf'.
What is the overall impression of these reviews? Give most prevalent examples in bullets. 
What do you suggest we focus on improving?
"""

review_chain = RetrievalQA.from_llm(llm=llm, retriever=vstore.as_retriever())
result=review_chain.run(q)
result


llama_print_timings:        load time =  1401.45 ms


'\nOverall, the reviews for the Premium Shemagh Head Neck Scarf are positive and highlight its versatility, lightweight design, and multiple uses. The product is also praised for its authenticity and quality compared to other similar products on the market. Some reviewers suggest that it can be a bit difficult to wash and dry, but this is not a prevalent issue among all reviews.\nBased on these reviews, one suggestion for improvement could be to provide more detailed information about the care and maintenance of the product, such as how to properly wash and dry it to maintain its quality and prevent shrinkage. Additionally, providing more specific information about the different colors available and their respective patterns could also be helpful for customers.'

llama_print_timings:      sample time =    61.94 ms /   156 runs   (    0.40 ms per run)
llama_print_timings: prompt eval time = 88428.79 ms /   492 tokens (  179.73 ms per token)
llama_print_timings:        eval time = 44105.34 ms /   155 runs   (  284.55 ms per run)
llama_print_timings:       total time = 140416.75 ms


In [48]:
# Rename columns in dataframe and create metadatafield in order to do upserts with Pinecone's Python client directly

df_review=df_review.rename(columns={'embeddings':'values','reviewerID':'id'})
df_review['metadata']=df_review.apply(lambda row: dict(rating=row['overall']), axis=1)

In [49]:
df_review.head(3)

,id,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,truncated,values,metadata
37582,A26QHY5FHZ1HGO,B004B8C2CQ,A. Ahmed,"[0, 0]","It is a great material, great color! and a goo...",5.0,"great material, great color!",1358640000,"01 20, 2013","It is a great material, great color! and a goo...","[0.012763723731040955, 0.05646521970629692, -0...",{'rating': 5.0}
37583,A1UHYD0RBA0AVM,B004B8C2CQ,aak,"[0, 0]",I purchased the blue and white colorway and I ...,5.0,Looks amazing,1344643200,"08 11, 2012",I purchased the blue and white colorway and I ...,"[0.005255925469100475, 0.05292578414082527, 0....",{'rating': 5.0}
37584,A1RC0SK3X55PQK,B004B8C2CQ,Aaron Don,"[0, 0]",It didn't feel like its supposed to be warm li...,3.0,yeah shemagh,1390867200,"01 28, 2014",It didn't feel like its supposed to be warm li...,"[-0.0018896874971687794, -0.030937310308218002...",{'rating': 3.0}


In [82]:
# Create two copies of data, one for the upsert and one for extracting reviewText from ids return from the filtered similarity search

data=df_review[['metadata','values','id']].to_dict(orient='records')
data_local=df_review[['metadata','values','reviewText','id']].to_dict(orient='records')

In [118]:
# Get the rating from id

get_rating_from_id = {
    x['id']: {
        'rating': x['metadata']['rating'],
        'review': x['reviewText'],
    } for x in data_local}

## Use vector database to filter rate and do query.
### Option 1. Use Pinecone

In [59]:
# Create the Pinecone index

pinecone.create_index(name='filtered', metric='euclidean', dimension=768)
index = pinecone.Index('filtered')

In [60]:
# Upload the data in batches of 50

from tqdm.auto import tqdm

for i in tqdm(range(0, len(data), 50)):
    j= i + 50
    if j > len(data):
        j = len(data)
    batch = data[i: j]
    index.upsert(vectors=batch)

  0%|          | 0/15 [00:00<?, ?it/s]

In [115]:
# Python function that retrieves reviews matching query and specific rate

def review_and_rating(query,rating):
    query=embeddings.embed_query(query)
    results = index.query(queries=[query], top_k=100, filter={'rating': {'$eq': rating}})
    ids = [i['id'] for i in results['results'][0]['matches']]
    l=[]
    for i in ids:
        l.append(get_rating_from_id[i])
    return pd.DataFrame(l)

In [ ]:
# Repurchase list and winback list

repurchase_list=review_and_rating('will purchase again', 5.0)

winback=review_and_rating('disappointed', 1.0)

### Option 2. Use FAISS

In [99]:
# Create FAISS index
import faiss

vector_list = [item['values'] for item in data]
vector_np = np.array(vector_list, dtype=np.float32)
vector_dim = vector_np.shape[1]

index = faiss.IndexFlatL2(vector_dim)
index.add(vector_np)
print("Number of vectors in the index:", index.ntotal, ". The dimension is:", vector_dim)

Number of vectors in the index: 719 . The dimension is: 768


In [120]:
# Run a filtered similarity search

query=embeddings.embed_query("will buy again")
query_np = np.array([query], dtype=np.float32)
results = index.search(query_np, k=100)
print(results, len(results))

(array([[1.1885645, 1.2211387, 1.3141928, 1.3230431, 1.327287 , 1.3280946,
        1.3313303, 1.3382936, 1.3405907, 1.3422668, 1.3560793, 1.3606681,
        1.3693402, 1.3706024, 1.371312 , 1.3746797, 1.3812767, 1.3879317,
        1.3994509, 1.4009572, 1.4066344, 1.4076196, 1.4100566, 1.4131273,
        1.4145195, 1.4170316, 1.4267899, 1.4285368, 1.4297657, 1.4303744,
        1.4363182, 1.439378 , 1.4439751, 1.4551479, 1.4553006, 1.4554669,
        1.4560657, 1.456203 , 1.4569952, 1.4602433, 1.4619151, 1.462318 ,
        1.4645083, 1.4656717, 1.470614 , 1.4713892, 1.4721059, 1.4727825,
        1.4733442, 1.4748217, 1.475667 , 1.4766885, 1.4786202, 1.4793388,
        1.4796015, 1.4797572, 1.4802284, 1.483944 , 1.4867296, 1.4876947,
        1.4892585, 1.4894409, 1.4896647, 1.4931965, 1.4938257, 1.4948148,
        1.4954383, 1.49684  , 1.4981658, 1.5000322, 1.5022714, 1.5044751,
        1.5054857, 1.5067017, 1.5080144, 1.5093617, 1.5122651, 1.5146686,
        1.5155785, 1.519015 , 1.51928

In [124]:
# Python function that retrieves reviews matching query and specific rate

def review_and_rating(query, rating):
    query=embeddings.embed_query(query)
    # Define a query vector
    query_np = np.array([query], dtype=np.float32)
    # Perform a search for the k nearest neighbors
    distances, indices = index.search(query_np, 100)
    # Filter the search results based on rating
    ids = []
    
    for idx, distance in zip(indices[0], distances[0]):
        if data[idx]['metadata']['rating'] == rating:
            ids.append(data[idx]['id'])

    l=[]
    for i in ids:
        l.append(get_rating_from_id[i])
    return pd.DataFrame(l)

In [132]:
# Repurchase list and winback list

repurchase_list=review_and_rating('will purchase again', 5.0)
winback=review_and_rating('disappointed', 1.0)

print(repurchase_list)
print(winback)

    rating                                             review
0      5.0  very happy with my purchase. it was exactly wh...
1      5.0  Bought 2 so far, good looking and they hold th...
2      5.0  This is an excellent value!  I received the pr...
3      5.0  I just bought this past month and i cant ask f...
4      5.0  Looks awesome, feels awesome, was exactly as p...
..     ...                                                ...
69     5.0  I am thrilled with the fabric and look of this...
70     5.0  i love the feel of the clothing, the color is ...
71     5.0  I bought mine in the dark brown and black colo...
72     5.0  Just what I wanted. Great quality product that...
73     5.0  Yes cant beat these for the price plus everyon...

[74 rows x 2 columns]
   rating                                             review
0     1.0  I recently ordered six of these and quite fran...
1     1.0  The light green scarf I ordered finally showed...
2     1.0  This is a huge let down and disappointm